Question 3: Data Wrangling - Duplicate Transactions

1. Can you programmatically identify reversed and multi-swipe transactions?

2. What total number of transactions and total dollar amount do you estimate for the reversed transactions?
For the multi-swipe transactions? (please consider the first transaction to be "normal" and exclude it from the number of transaction and dollar amount counts) 

3. Did you find anything interesting about either kind of transaction?

In [1]:
import sys
import warnings
warnings.filterwarnings("ignore")
sys.path.append('utils')
import pandas as pd
from loader_utils import url_dframe
from EDA_utils import *
from datetime import timedelta
from wrangling_utils import transaction_classification

In [2]:
weblink = "https://github.com/CapitalOneRecruiting/DS/raw/master/transactions.zip"
cap_df = url_dframe(weblink)
cap_df.replace('', np.nan, inplace=True)
cap_df.head(5)

,accountNumber,customerId,creditLimit,availableMoney,transactionDateTime,transactionAmount,merchantName,acqCountry,merchantCountryCode,posEntryMode,...,echoBuffer,currentBalance,merchantCity,merchantState,merchantZip,cardPresent,posOnPremises,recurringAuthInd,expirationDateKeyInMatch,isFraud
0,737265056,737265056,5000,5000.0,2016-08-13T14:27:32,98.55,Uber,US,US,02,...,NaN,0.0,NaN,NaN,NaN,False,NaN,NaN,False,False
1,737265056,737265056,5000,5000.0,2016-10-11T05:05:54,74.51,AMC #191138,US,US,09,...,NaN,0.0,NaN,NaN,NaN,True,NaN,NaN,False,False
2,737265056,737265056,5000,5000.0,2016-11-08T09:18:39,7.47,Play Store,US,US,09,...,NaN,0.0,NaN,NaN,NaN,False,NaN,NaN,False,False
3,737265056,737265056,5000,5000.0,2016-12-10T02:14:50,7.47,Play Store,US,US,09,...,NaN,0.0,NaN,NaN,NaN,False,NaN,NaN,False,False
4,830329091,830329091,5000,5000.0,2016-03-24T21:04:46,71.18,Tim Hortons #947751,US,US,02,...,NaN,0.0,NaN,NaN,NaN,True,NaN,NaN,False,False


In [3]:
print('Type of Transaction:', cap_df['transactionType'].unique())

Type of Transaction: ['PURCHASE' 'ADDRESS_VERIFICATION' 'REVERSAL' nan]


##### Step1: DataFrame is sorted by accountNumber and within each account, it is further sorted by transactionDateTime
##### Step2: Group the DataFrame to have the same accountNumber and transactionAmount
##### Step3: Apply transaction_classification function to classify a group of transactions and return a summary based on the number and type of transactions
##### Step4:  Filter and compute the total number of transactions and total amount for transactions categorized as 'Reversal', adding a prefix Reversal_
##### Step4:  Filter and compute the total number of transactions and total amount for transactions categorized as 'Multi-swipe', adding a prefix Multi_swipe_

In [4]:
cap_df['transactionDateTime'] = pd.to_datetime(cap_df['transactionDateTime'], errors='coerce')
df_sorted = cap_df.sort_values(['accountNumber', 'transactionDateTime'])
grouped = df_sorted.groupby(['accountNumber', 'transactionAmount'])
result = grouped.apply(transaction_classification).reset_index()

reversal_stats = result[result['category'] == 'Reversal'].agg({
    'count': 'sum',
    'amount': 'sum'
}).add_prefix('Reversal_')

multi_swipe_stats = result[result['category'] == 'Multi-swipe'].agg({
    'count': 'sum',
    'amount': 'sum'
}).add_prefix('Multi_swipe_')

In [5]:
print("Reversal Transactions:")
print(f"Total Number of Transaction: {reversal_stats['Reversal_count']}")
print(f"Total amount: ${reversal_stats['Reversal_amount']:.2f}")
print("\nMulti-swipe Transactions:")
print(f"Total Number of Transaction: {multi_swipe_stats['Multi_swipe_count']}")
print(f"Total amount: ${multi_swipe_stats['Multi_swipe_amount']:.2f}")

Reversal Transactions:
Total Number of Transaction: 29830.0
Total amount: $2842573.14

Multi-swipe Transactions:
Total Number of Transaction: 72019.0
Total amount: $4124160.15


In [6]:
# Analyze Reversal Transactions
from collections import Counter
reversals = result[result['category'] == 'Reversal']
reversal_merchants = Counter(reversals['merchant'])
largest_reversal = reversals.loc[reversals['amount'].idxmax()]

# Analyze Multi-swipe Transactions
multi_swipes = result[result['category'] == 'Multi-swipe']
multi_swipe_merchants = Counter(multi_swipes['merchant'])
largest_multi_swipe = multi_swipes.loc[multi_swipes['amount'].idxmax()]

In [7]:
print("Interesting findings about Reversal Transactions:")
print(f"Number of unique merchants involved in Reversal Transactions: {len(reversal_merchants)}")
print(f"Merchant with most reversals: {reversal_merchants.most_common(1)[0][0]}", \
      'total of', reversal_merchants.most_common(1)[0][1], 'Transactions')
print(f"Largest reversal amount: ${largest_reversal['amount']:.2f} at {largest_reversal['merchant']}")

print("\nInteresting findings about Multi-swipe Transactions:")
print(f"Number of unique merchants involved in Multi-swipe Transaction: {len(multi_swipe_merchants)}")
print(f"Merchant with most multi-swipes: {multi_swipe_merchants.most_common(1)[0][0]}", \
      'total of', multi_swipe_merchants.most_common(1)[0][1], 'Transactions')
print(f"Largest multi-swipe amount: ${largest_multi_swipe['amount']:.2f} at {largest_multi_swipe['merchant']}")

Interesting findings about Reversal Transactions:
Number of unique merchants involved in Reversal Transactions: 1886
Merchant with most reversals: Lyft total of 622 Transactions
Largest reversal amount: $2069.37 at South Steakhouse #73819

Interesting findings about Multi-swipe Transactions:
Number of unique merchants involved in Multi-swipe Transaction: 2263
Merchant with most multi-swipes: Play Store total of 1053 Transactions
Largest multi-swipe amount: $3933.23 at MyApron
